In [1]:
import pandas as pd

In [2]:
'''
透视表(pivot table)是各种电子表格程序和其他数据分析软件中一种常见的数据汇总工具。
它根据一个或多个键对数据进行聚合，并根据行和列上得分组建将数据分配到各个矩形区域中。
在Python和pandas中，可以通过本章所介绍的groupby功能以及（能够利用层次化索引的）重塑运算制作透视表。
DataFrame有一个pivot_table方法，此外还有一个顶级的pandas.pivot_table函数。
除了能为groupby提供便利之外，pivot_table还可以添加分项小计（也叫margins）。
'''

'\n透视表(pivot table)是各种电子表格程序和其他数据分析软件中一种常见的数据汇总工具。\n它根据一个或多个键对数据进行聚合，并根据行和列上得分组建将数据分配到各个矩形区域中。\n在Python和pandas中，可以通过本章所介绍的groupby功能以及（能够利用层次化索引的）重塑运算制作透视表。\nDataFrame有一个pivot_table方法，此外还有一个顶级的pandas.pivot_table函数。\n除了能为groupby提供便利之外，pivot_table还可以添加分项小计（也叫margins）。\n'

In [3]:
tips = pd.read_csv('data/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

total_bill   tip     sex smoker  day    time  size   tip_pct
0       16.99  1.01  Female     No  Sun  Dinner     2  0.059447
1       10.34  1.66    Male     No  Sun  Dinner     3  0.160542
2       21.01  3.50    Male     No  Sun  Dinner     3  0.166587
3       23.68  3.31    Male     No  Sun  Dinner     2  0.139780
4       24.59  3.61  Female     No  Sun  Dinner     4  0.146808

In [4]:
# 平均值，使用group_by
tips.groupby(['sex', 'smoker']).mean()

total_bill       tip      size   tip_pct
sex    smoker                                          
Female No       18.105185  2.773519  2.592593  0.156921
       Yes      17.977879  2.931515  2.242424  0.182150
Male   No       19.791237  3.113402  2.711340  0.160669
       Yes      22.284500  3.051167  2.500000  0.152771

In [5]:
# 平均值，使用pivot_table
tips.pivot_table(index=['sex', 'smoker'])

size       tip   tip_pct  total_bill
sex    smoker                                          
Female No      2.592593  2.773519  0.156921   18.105185
       Yes     2.242424  2.931515  0.182150   17.977879
Male   No      2.711340  3.113402  0.160669   19.791237
       Yes     2.500000  3.051167  0.152771   22.284500

In [14]:
# 现在假设我们只想聚合tip_pct和size，而且想根据day进行分组。我将smoker放到列上，把day放到行上。
tips.pivot_table(values=['tip_pct', 'size'], index=['sex', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
sex    day                                         
Female Fri   2.500000  2.000000  0.165296  0.209129
       Sat   2.307692  2.200000  0.147993  0.163817
       Sun   3.071429  2.500000  0.165710  0.237075
       Thur  2.480000  2.428571  0.155971  0.163073
Male   Fri   2.000000  2.125000  0.138005  0.144730
       Sat   2.656250  2.629630  0.162132  0.139067
       Sun   2.883721  2.600000  0.158291  0.173964
       Thur  2.500000  2.300000  0.165706  0.164417

In [ ]:
'''
还可以对这个表作进一步处理，传入margins=True添加加分小计。
这将会添加标签为ALL的行和列，其值对应于单个等级中所有数据的分组统计。
在下面这个例子中，ALL值为平均数：不单独考虑烟民与非烟民（ALL列），不单独考虑行分组两个级别中的任何单项（ALL行）。
换句话说：就是下面的ALL行和右边的ALL列只统计对应的列和行。
'''

In [15]:
tips.pivot_table(values=['tip_pct', 'size'], index=['sex', 'day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
sex    day                                                             
Female Fri   2.500000  2.000000  2.111111  0.165296  0.209129  0.199388
       Sat   2.307692  2.200000  2.250000  0.147993  0.163817  0.156470
       Sun   3.071429  2.500000  2.944444  0.165710  0.237075  0.181569
       Thur  2.480000  2.428571  2.468750  0.155971  0.163073  0.157525
Male   Fri   2.000000  2.125000  2.100000  0.138005  0.144730  0.143385
       Sat   2.656250  2.629630  2.644068  0.162132  0.139067  0.151577
       Sun   2.883721  2.600000  2.810345  0.158291  0.173964  0.162344
       Thur  2.500000  2.300000  2.433333  0.165706  0.164417  0.165276
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [16]:
# 要使用其他的聚合函数，将其传给参数aggfunc即可。例如，使用count或len可以得到有关分组大小的交叉表。
tips.pivot_table('tip_pct', index=['sex', 'smoker'], columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
sex    smoker                               
Female No       2.0  13.0  14.0  25.0   54.0
       Yes      7.0  15.0   4.0   7.0   33.0
Male   No       2.0  32.0  43.0  20.0   97.0
       Yes      8.0  27.0  15.0  10.0   60.0
All            19.0  87.0  76.0  62.0  244.0

In [ ]:
# 交叉表(cross-tabulation, 简称crosstab)是一种用于计算分组频率的特殊透视表。

In [7]:
data = pd.DataFrame({'Sample': range(1, 11), 
                     'Gender': ['Female', 'Male', 'Female', 'Male', 'Male', 'Male', 'Female', 'Female', 'Male', 'Female'], 
                    'Handedness': ['Right-handed', 'Left-handed', 'Right-handed', 'Right-handed', 'Left-handed', 'Right-handed', 'Right-handed', 'Left-handed', 'Right-handed', 'Right-handed']})
data

Gender    Handedness  Sample
0  Female  Right-handed       1
1    Male   Left-handed       2
2  Female  Right-handed       3
3    Male  Right-handed       4
4    Male   Left-handed       5
5    Male  Right-handed       6
6  Female  Right-handed       7
7  Female   Left-handed       8
8    Male  Right-handed       9
9  Female  Right-handed      10

In [13]:
# 使用pivot_table
data.pivot_table(index=['Gender'], columns='Handedness', aggfunc='mean', margins=True)

Sample                 
Handedness Left-handed Right-handed All
Gender                                 
Female               1            4   5
Male                 2            3   5
All                  3            7  10

In [20]:
# 使用cross_table
pd.crosstab(data.Gender, data.Handedness, margins=True)

Handedness  Left-handed  Right-handed  All
Gender                                    
Female                1             4    5
Male                  2             3    5
All                   3             7   10

In [21]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244